In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
from time import perf_counter
import glob
import json
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib.pyplot import figure
import tweets_utils
import importlib
%autosave 3600

Autosaving every 3600 seconds


In [4]:
files_users = glob.glob("F:/COVID-19-Tweets/*/users.csv")
files_tweets = glob.glob("F:/COVID-19-Tweets/*/tweets.csv")

In [4]:
names = []
users = []
verified = []
cols_users = [u'id', u'screen_name', u'verified'] 

cols_tweets = [
    u'user_id',u'user_screen_name', u'hashtags', u'urls',
    u'rt_user_screen_name', u'rt_user_id', 
    u'in_reply_to_screen_name', u'in_reply_to_user_id', 
    u'rt_created_at', u'in_reply_to_status_id'
              ]

In [3]:
chunksize = 10
for chunk in pd.read_csv(files_tweets[0], chunksize=10, lineterminator = '\n'):
    df = chunk
    break
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10 non-null     int64  
 1   id_str                     10 non-null     int64  
 2   created_at                 10 non-null     object 
 3   user_id                    10 non-null     int64  
 4   user_screen_name           10 non-null     object 
 5   user_verified              10 non-null     bool   
 6   user_created_at            10 non-null     object 
 7   text                       10 non-null     object 
 8   source                     10 non-null     object 
 9   coordinates                0 non-null      float64
 10  is_quote_status            10 non-null     bool   
 11  retweet_count              10 non-null     int64  
 12  favorite_count             10 non-null     int64  
 13  favorited                  10 non-null     bool   
 1

In [17]:
result = []
importlib.reload(tweets_utils)
start_time = time.perf_counter()
j = 0
for filename in files_users:
    print(f"Reading file: {filename}")
    result.extend(tweets_utils.process_all_data(filename, cols_users, False))
    lista = result[j]
    j = j + 1
    print(f"Mese numero {j}")
    for i in range(len(lista)):
        users.extend(lista[i]["users"])
        verified.extend(lista[i]["verified"])
        
for i in range(len(users)):
    if(verified[i] == True):
        names.append(users[i])
stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

Reading file: F:/COVID-19-Tweets\2020-01\users.csv
Processing chunk 1
Processing chunk 2
Mese numero 1
Reading file: F:/COVID-19-Tweets\2020-02\users.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Mese numero 2
Reading file: F:/COVID-19-Tweets\2020-03\users.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Mese numero 3
Reading file: F:/COVID-19-Tweets\2020-04\users.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Mese numero 4
Reading file: F:/COVID-19-Tweets\2020-05\users.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Mese numero 5
Time:  238.7169179


In [18]:
names = list(pd.Series(names).unique())
names

['MichaelDClark1',
 'SBengali',
 'dzbb',
 'Reuters',
 'fox6now',
 'PaulRieckhoff',
 'shongables',
 'samdastyari',
 'RANDCorporation',
 'NZStuff',
 'Bisniscom',
 'mph_for_doctors',
 'SilkyGilkey',
 'danschoenmn',
 'business',
 'staronline',
 'bhairavigoswami',
 'lauriechenwords',
 'Jan_Achakzai',
 'DiMartinoBooth',
 'MetroTVToday',
 'selebrita7',
 'tempodotco',
 '4029news',
 '7NewsAustralia',
 'JChengWSJ',
 'IBexWeBex',
 'KrutikaKuppalli',
 'infobae',
 'ABSCBNNews',
 'VancouverSun',
 'YahooSG',
 'DrJenCaudle',
 'republikaonline',
 'anuraag_saxena',
 'suhasinih',
 'mediaindonesia',
 'SHEIN_official',
 'Aura_',
 'dailytelegraph',
 'htTweets',
 'DDNewsGujarati',
 'MothershipSG',
 'kapookdotcom',
 'dhpierson',
 'AjayFry',
 'SarahTitterton',
 'maladarmadi',
 'DM_Bhadrak',
 'JunShimoyama',
 'ktnewsonline',
 'HindustanTimes',
 'alex',
 'oatila',
 '501Awani',
 'ians_india',
 'jawapos',
 'Ch7HDNews',
 'WilliamYang120',
 'CCTVAsiaPacific',
 'Daily_Express',
 'SBSNews',
 'SuNoticiero',
 'leeminjeo

In [24]:
result = []
data = pd.DataFrame()
importlib.reload(tweets_utils)
start_time = time.perf_counter()
j = 0
for filename in files_tweets:
    print(f"Reading file: {filename}")
    result.extend(tweets_utils.process_all_data(filename, cols_tweets, True, names))
    lista = result[j]
    j = j + 1
    print(f"Mese numero {j}")
    for i in range(len(lista)):
        data = data.append(lista[i]["df"])
    print(len(data))
stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

Reading file: F:/COVID-19-Tweets\2020-01\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Mese numero 1
113563
Reading file: F:/COVID-19-Tweets\2020-02\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Mese numero 2
416405
Reading file: F:/COVID-19-Tweets\2020-03\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Mese numero 3
743542
Reading file: F:/COVID-19-Tweets\2020-04\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 

In [26]:
data = data.reset_index(drop=True)
data

,user_id,user_screen_name,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,rt_created_at,rt_user_id,rt_user_screen_name,hashtags,urls
0,4.260338e+08,HHSRegion8,NaN,NaN,NaN,Tue Jan 21 20:57:06 +0000 2020,1.465700e+08,CDCgov,"[{'text': 'coronavirus', 'indices': [47, 59]}]",[]
1,3.087536e+08,SmithDanaG,NaN,NaN,NaN,Tue Jan 21 22:37:07 +0000 2020,4.693387e+07,RobertRoyBritt,"[{'text': 'coronavirus', 'indices': [75, 87]}]",[]
2,5.883524e+08,MelissaFox26,NaN,NaN,NaN,NaN,NaN,NaN,[],"[{'url': 'https://t.co/dkr9py0rqT', 'expanded_..."
3,1.754954e+08,CDCtravel,NaN,NaN,NaN,NaN,NaN,NaN,"[{'text': 'China', 'indices': [47, 53]}, {'tex...","[{'url': 'https://t.co/7aUPsJWZaV', 'expanded_..."
4,2.186825e+09,AlainnaJJ,NaN,NaN,NaN,Tue Jan 21 23:08:09 +0000 2020,2.802272e+07,carlynharris,[],[]
...,...,...,...,...,...,...,...,...,...,...
1687338,6.073330e+08,WVGovernor,NaN,NaN,NaN,NaN,NaN,NaN,"[{'text': 'COVID19', 'indices': [114, 122]}, {...","[{'url': 'https://t.co/Ad33fbC98s', 'expanded_..."
1687339,1.392342e+09,NorfolkPD,NaN,NaN,NaN,NaN,NaN,NaN,"[{'text': 'NorfolkPD', 'indices': [0, 10]}, {'...","[{'url': 'https://t.co/4uYlwOniB6', 'expanded_..."
1687340,1.580251e+09,IMG_ST,NaN,NaN,NaN,Wed May 27 11:54:10 +0000 2020,7.810246e+17,mwsachsenanhalt,"[{'text': 'Wirtschaft', 'indices': [109, 120]}]",[]
1687341,5.225004e+08,shond3,NaN,NaN,NaN,NaN,NaN,NaN,[],"[{'url': 'https://t.co/QrEm6uQWvb', 'expanded_..."


In [5]:
path =  glob.glob("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/large_files/")
path

['C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/large_files/']

In [27]:
data.to_csv("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/large_files/verified_tweets.csv", index=False)

In [17]:
cols = [
    u'id',u'user_id',u'user_screen_name',u'urls',
    u'rt_user_screen_name', u'rt_user_id', 
    u'in_reply_to_screen_name', u'in_reply_to_user_id']

In [39]:
ccdh_rt_complete = pd.read_csv(path[0] + "ccdh_retweets.csv", usecols = cols)
ccdh_rt_complete

,id,user_id,user_screen_name,in_reply_to_user_id,in_reply_to_screen_name,rt_user_id,rt_user_screen_name,urls
0,1424933544229580812,1298607930800054273,Luis40461853,NaN,NaN,1.252452e+07,mercola,[]
1,1424914837163892736,30849514,Dorunda,NaN,NaN,1.252452e+07,mercola,"[{'url': 'https://t.co/bO3BY3jZUa', 'expanded_..."
2,1424934389272678401,512506170,richardfinnman,NaN,NaN,1.252452e+07,mercola,[]
3,1424898609414291456,2675586505,psipika,NaN,NaN,1.252452e+07,mercola,[]
4,1424982702378287109,1417658669840273413,BharatT60678892,NaN,NaN,1.252452e+07,mercola,[]
...,...,...,...,...,...,...,...,...
103734,1420425173455491073,1017797970115297285,maxine_oxford,NaN,NaN,1.152076e+18,Lilymackay19,[]
103735,1420425712985587715,4201367609,DrJacks_sagt,NaN,NaN,3.378086e+08,RobertKennedyJr,[]
103736,1420425656760942602,797829335755407362,wgamzatov,NaN,NaN,3.378086e+08,RobertKennedyJr,[]
103737,1420425239088013322,1229081670382489602,Styndall11,NaN,NaN,3.378086e+08,RobertKennedyJr,[]


In [8]:
ccdh_rt_searched = pd.read_csv(path[0] + "ccdh_user_rt.csv")
ccdh_rt_searched

,id,user_id,name,rt_id,rt_name,rt_url
0,1220056186927468544,1.104760e+18,Exvaxxer_Mommy,18093097.0,unhealthytruth,[]
1,1220061563220103174,1.055931e+18,patriot7842,18093097.0,unhealthytruth,[]
2,1220062091463331847,2.282256e+07,dihoppy,18093097.0,unhealthytruth,[]
3,1220058941553463297,4.182128e+07,liteseeker,18093097.0,unhealthytruth,[]
4,1220059473626173441,7.150825e+08,Heidi559Heidi,18093097.0,unhealthytruth,[]
...,...,...,...,...,...,...
3524,1262521533919969280,1.163569e+18,Nomoresilence20,337808606.0,RobertKennedyJr,[]
3525,1265359228144771072,1.082425e+18,Warrior17761,35013035.0,BusyDrT,[]
3526,1263475020321431552,8.863322e+06,TonyGosling,337808606.0,RobertKennedyJr,[]
3527,1259382911201169408,1.206716e+18,banka_beata,35013035.0,BusyDrT,[]


In [31]:
for i in ccdh_rt_complete["id"]:
    for j in ccdh_rt_searched["id"]:
        if str(i) == str(j):
            print(i)
            break

KeyboardInterrupt: 

In [32]:
check = any(item in ccdh_rt_complete["id"] for item in ccdh_rt_searched["id"])

In [34]:
type(ccdh_rt_complete["id"][0])

numpy.int64

In [36]:
type(ccdh_rt_searched["id"][0])

numpy.int64